# 1. Constant-folding benchmark

Run micro-benchmark via
`tuplex-public/tuplex/build$ ctest -R BasicInvocation.MicroConstantFoldedAggregation --verbose`


What?
Goal of this benchmark is to estimate how much benefit comes from replacing a hashmap within aggregateByKey with aggregate when the key is constant.

Setup:

N rows, 3 columns of int64_t integers. First column year, second column month, third column data to sum up.

I.e., corresponds to query

```
ctx.csv(input_pattern)
        .selectColumns(std::vector<std::string>({"YEAR", "MONTH", "ARR_DELAY"}))
        .aggregateByKey(UDF("lambda a, b: a + b"),
                        UDF("lambda a, row: a + row['ARR_DELAY']"),
                        Row(0), std::vector<std::string>({"YEAR", "MONTH"}))
        .show();
```

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/bbsn00/constant-folded-agg.csv')

In [8]:
gdf = df.groupby(['mode', 'N']).agg(['mean', 'std'])['time_in_s'] \
       .reset_index().sort_values(by='mean').reset_index(drop=True)
gdf

,mode,N,mean,std
0,constant-folded,1000000,0.006313,0.000525
1,hashmap,1000000,0.065912,0.000229


In [14]:
# speedup factor
speedup_factor = gdf.loc[1, 'mean'] / gdf.loc[0, 'mean']
speedup_factor

10.441187349658588